## 1  Generating (Shakespearean) Text with a GPT-like Transformer

In this exercise we are going to build a GPT-like transformer. Such a transformeris a decoder-only transformer and hence the doesn’t include a cross attentionlayer.

As a practical application of the transformer, we will train it to generate Shake-spearean text.

**Note**: the following video1from Andrej Karpathy explains in detail how to builda GPT-like decoder. In the video the Pytorch framework is used, but the con-cepts are identical. The video also follows the “Attention is all you need” paper,apart from the placement of the *LayerNormalization* layers.

The overall architecture of a **decoder-only transformer** is shown in Figure 1

### 1.1  Implement the FeedForward Layer

We start by implementing theFeedForwardlayer. According to equation (2) ofthe “Attention is all you Need” paper, this layer performs the following calculation:
$$
\text{FFN}(𝑥) = \text{max}(0, xW1+ b1)W_2+ b_2
$$
which is applied to each position (i.e. each time step) $𝑥$ independently. The dimensions of the input and output are identical, but the layer in between is 4 times wider.

For this exercise, we are going to implement this layer as a subclass ofkeras.layers.Layerbut we will **not** use any other layers, instead you should [https://www.youtube.com/watch?v=kCc8FmEb1nY2](https://www.youtube.com/watch?v=kCc8FmEb1nY2)

![Figure 1: Architech )f a decoder only transformer](../img/fig1_architechture_of_a_decoder-only_transformer.png)

In [5]:
%uv install --upgrade pip --quiet
%uv install keras --quiet
%uv install tensorflow-metal --quiet
%uv install tensorflow-macos --quiet

UsageError: Line magic function `%uv` not found.


In [6]:
%pip install keras
%pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 739.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 103.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 122.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 128.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00


In [7]:
import tensorflow as tf
import keras

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [18]:
# Simple FeedForward layer
@keras.saving.register_keras_serializable()
class FeedForward(keras.layers.Layer):

    def __init__(self, factor=4, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor

    def build(self, batch_input_shape):
        time_steps, embed_size = batch_input_shape[1:]
        #! YOUR CODE HERE:
        self.w1 = self.add_weight(shape=(embed_size, self.factor * embed_size))
        self.w2 = self.add_weight(shape=(self.factor * embed_size, embed_size))
        self.b1 = self.add_weight(shape=(self.factor * embed_size,))
        self.b2 = self.add_weight(shape=(embed_size,))

    #? Call kun je oproepen met `FeedForward()()`
    def call(self, inputs):
        #! YOUR CODE HERE:
        #! Perform calculation on inputs and return result
        inputs = keras.ops.matmul(inputs,self.w1)
        inputs = inputs + self.b1
        inputs = keras.layers.Activation("relu")(inputs)

        inputs = keras.ops.matmul(inputs,self.w2)
        inputs = inputs + self.b2
        return inputs

    def get_config(self):
        base_config = super().get_config()
        return{**base_config,"factor": self.factor,}

Use the code below to perform a basic test of your layer:

In [17]:
# Simple test code
TEST_SHAPE = (2, 10, 32) # Batchsize2,10 timesteps, embedding dimension 32

X = keras.random.normal(shape=TEST_SHAPE)
ff = FeedForward()
print(f"Shape of output {ff(X).shape}") # Should print (2,10,32)

for w in ff.get_weights(): # Check that the shapes are what you expect
    print(w.shape)

Shape of output (2, 10, 32)
(32, 128)
(128, 32)
(128,)
(32,)


1.2  Implement a GPT Decoder Block

Next, implement a *GPTDecoderBlock* class as a subclass of *keras.layers.Layer*. This layer represents one decoder block. It consists of
- Masked (or causal) multi-head attention.
- Layer normalization (and a skip connection)
- A feed forward layer (which was implemented in the previous step)
- A second layer normalization step (and a skip connection)

You can see the starter code for *GPTDecoderBlock* class in Figure 3.

Note the following:
1. Keras provides a *MultiHeadAttention* attention class that you can use.In the “Attention is all you Need” paper it is mentioned at the end of sec-tion 3.2.2 that the dimensions for the keys and values are
   $$
   d_k = d_v = d_{model}/h,
   $$
   where $h$ denotes the number of heads and $𝑑_{model}$ is the dimension of the embeddings.
2. In thecallmethod you need to make sure to apply the causal masking.
3. In section 3.1 of the “Attention is all you Need” paper you can see that the skip connection and the layer normalisation are implemented as follows:
   $$
   \text{LayerNorm}(𝑥 +\text{Sublayer}(𝑥)),
   $$
If you don’t, the model will seem to learn very quickly but at test time it will not doanything useful.

In [8]:
# A GPT decoderblock (with out crossattention)
@keras.saving.register_keras_serializable()
class GPTDecoderBlock(keras.layers.Layer):
    def __init__(self, num_heads, embed_size, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.embed_size = embed_size
        #! YOUR CODE HERE
        #! Add needed layers (either from Keras or your own custom layer)
        self.attention = keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                         key_dim=(embed_size//num_heads))
        self.normalization_1 = keras.layers.LayerNormalization()
        self.normalization_2 = keras.layers.LayerNormalization()
        self.feed_forward_network = FeedForward()

    def call(self, inputs):
        #! YOUR CODE HERE
        #! Perform the computation on inputs and return result
        skip_connection_1 = inputs  # (seq_length, embed_size)

        # * Output van attention
        x = self.attention(inputs, inputs, use_causal_mask=True)
        # * Output van attention en normalisatie
        x = self.normalization_1(x + skip_connection_1)

        skip_connection_2 = x

        # * Output van feedforward netwerk
        x = self.feed_forward_network(x)
        # * Output van feedforward en normalisatie
        x = self.normalization_2(x + skip_connection_2)
        return x  # (seq_length, embed_size)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config,
                "num_heads": self.num_heads,
                "embed_size": self.embed_size
                }

where Sublayer is either the multi-head attention layer or the feed forwardlayer.

Run the following code to perform a simple test of your class:

In [19]:
# Simple test code
TEST_SHAPE = (2, 10, 32) # Batchsize 2,10 time steps,embedding dimension 32

#? Wat is mijn context window (aka length)? 10
X = keras.random.normal(shape=TEST_SHAPE)
gpt_block = GPTDecoderBlock(num_heads=4, embed_size=32)
print(f"Shape of output {gpt_block(X).shape}") # Should print (2,10,32)

Shape of output (2, 10, 32)


## 1.3  Implement a EmbeddingWithPositionLayer

The decoder starts with an embedding layer which embeds the (integer) tokensinto a vector space. Next, a positional embedding is added to the token em-beddings. In the “Attention is all you Need” paper it is mentioned that theseembeddings can either be “fixed”, or they can be learned.

We will implement a class *EmbeddingWithPosition* which combines the tokenembedding with a learnable positional embedding. We willnotmake use of theEmbeddinglayer to implement the class. Implementing this “by hand” will helpyou gain a better understanding of what embeddings are actually doing.Start from the code in Figure4to implement this class.

Hints:
1. In thebuildmethod you should add two learnable weights matrices. The dimensions of these matrices depend on the values of the arguments thatwere passed to the constructor:
    - num_tokens: the number of tokens in the vocabulary
    - max_seq_length: the maximum length of any sequence. The model will not work if sequences with a length longer than this maximumlength are used
    - embed_size: the dimension of the embeddings.
2. In the `call` method, you can use *keras.ops.take* to select rows fromthe embedding matrix.
3. The positional embeddings are (by definition) the firstlengthrows fromthe positional embedding matrix.
4. Rely on the `+` operator to perform the broadcasting between the tokenembeddings and the positional embeddings

In [9]:
@keras.saving.register_keras_serializable()
class EmbeddingWithPosition(keras.layers.Layer):
    def __init__(self, num_tokens, max_seq_length, embed_size, **kwargs):
        super().__init__(**kwargs)
        #! YOUR CODE HERE
        #! Save constructor arguments
        self.num_tokens = num_tokens
        self.max_seq_length = max_seq_length
        self.embed_size = embed_size
    def build(self, batch_input_shape):
        print(f"Building EmbeddingWithPosition with input shape {batch_input_shape}")
        #! Shape not actually needed!!
        #! YOUR CODE HERE
        #! Add the weights for the two embeddings
        #? Token kunnen omzetten naar een embedding?
        #? token 0 (the)
        #? --> embedding [30,45,29,..., 223,45] # 512
        #? token 2 (or)
        #? --> embedding [12,34,56,...,78] # 512
        #? [
        #? (0): [30,45,29,...,223,45],
        #? ...
        #? (2): [12,34,56,...,78]
        #? ]
        self.embedding_loop_table = self.add_weight(shape=(self.num_tokens,self.embed_size))
        self.position_lookup_table = self.add_weight(shape=(self.max_seq_length,self.embed_size))
    def call(self, inputs):
        _, length = keras.ops.shape(inputs)
        #? YOUR CODE HERE
        #? Get both embeddings and add them.
        token_embeddings = keras.ops.take(self.embedding_loop_table,inputs,axis=0)
        position_embeddings = self.position_lookup_table[:length]
        return token_embeddings + position_embeddings

    def get_config(self):
        base_config = super().get_config()
        return{**base_config,
                "num_tokens": self.num_tokens,
                "max_seq_length": self.max_seq_length,
                "embed_size": self.embed_size}

In [21]:
tokens = keras.ops.convert_to_tensor([[1,3,5],[0,2,4]])
embed_layer = EmbeddingWithPosition(num_tokens=10,
                                    max_seq_length=5,
                                    embed_size=32)
print(embed_layer(tokens).shape) # Should print (2,3,32) -> batch size 2, 3 tokens, embedding size 32
for w in embed_layer.get_weights(): # Check that this is what you would expect
    print(w.shape)

Building EmbeddingWithPosition with input shape (2, 3)
(2, 3, 32)
(10, 32)
(5, 32)


## 1.4  Build the Complete Model

Write a methodget_modelthat returns a complete GPT-like decoder. Since wehave all the necessary layers, this is now a simple sequential model. Completethe code in Figure 5.

As you can see in Figure1, there is a linear layer after the last decoder block.This linear layer works independently for each token. For this exercise we willoutput the logits for the tokens instead of the token probabilities. Stated other-wise, the last layer in our model does not include an activation function

In [22]:
# Create a Model
def get_model(n_tokens, embed_size,num_blocks, num_heads,max_seq_length):
    model = keras.Sequential()
    #? Voor nu op None zetten
    model.add(keras.layers.Input(shape=(None,), dtype=tf.int32))
    # YOUR CODE HERE
    # Add positional embedding layer
    model.add(EmbeddingWithPosition(n_tokens, max_seq_length, embed_size))

    for i in range(num_blocks):
        # YOUR CODE HERE
        # Add decoder blocks
        model.add(GPTDecoderBlock(num_heads=num_heads,
                                  embed_size=embed_size,
                                  name=f"gpt_decoder_block_{i}"))

    # YOUR CODE HERE
    # Add output layer.
    # Output logits, not probabilities
    model.add(keras.layers.Dense(units=n_tokens,
                                 activation="softmax",name="output_layer"))

    return model

In [23]:
tokens = keras.ops.convert_to_tensor([[1,3,5],[0,2,4]])
model = get_model(n_tokens=10,
                  embed_size=32,
                  num_blocks=10,
                  num_heads=4,
                  max_seq_length=5)
print(model(tokens).shape) # Should print (2,3,10) -> batch size

Building EmbeddingWithPosition with input shape (None, None)
(2, 3, 10)


## 1.5  Preparing the Data for Shakespeare Text Generation.
1. Use the code from page 579 in the book to fetch the data:

In [24]:
shakespeare_url ="https://homl.info/shakespeare" #shortcut URL
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

2. Use a `keras.layers.TextVectorization` layer to encode this text. Split on characters, keep punctuation as well as upper- and lowercase letters

In [25]:
# Example code from the book Needs changing!!
#? Splitten op karakter niveau, lowercase
print(help(keras.layers.TextVectorization))
text_vec_layer = keras.layers.TextVectorization(split="character",
                                                standardize="lower")

Help on class TextVectorization in module keras.src.layers.preprocessing.text_vectorization:

class TextVectorization(keras.src.layers.layer.Layer)
 |  TextVectorization(max_tokens=None, standardize='lower_and_strip_punctuation', split='whitespace', ngrams=None, output_mode='int', output_sequence_length=None, pad_to_max_tokens=False, vocabulary=None, idf_weights=None, sparse=False, ragged=False, encoding='utf-8', name=None, **kwargs)
 |
 |  A preprocessing layer which maps text features to integer sequences.
 |
 |  This layer has basic options for managing text in a Keras model. It
 |  transforms a batch of strings (one example = one string) into either a list
 |  of token indices (one example = 1D tensor of integer token indices) or a
 |  dense representation (one example = 1D tensor of float values representing
 |  data about the example's tokens). This layer is meant to handle natural
 |  language inputs. To handle simple string inputs (categorical strings or
 |  pre-tokenized strin

3. `shakespeare_text` is a string and adapt expects a dataset or list

In [26]:
# Example code from the book. Can be used as is.
text_vec_layer.adapt([shakespeare_text])
"""
['', '[UNK]', ' ', 'e', 't', 'o', 'a', 'i', 'h', 's', 'r', 'n', '\n', 'l', 'd', 'u', 'm', 'y', 'w', ',', 'c', 'f', 'g', 'b', 'p', ':', 'k', 'v', '.', "'", ';', '?', '!', '-', 'j', 'q', 'x', 'z', '3', '&', '$']
"""
encoded = text_vec_layer([shakespeare_text])[0]

4. Print the vocabulary created from shakespeare_text

In [13]:
vocab = text_vec_layer.get_vocabulary()
print(vocab)

['', '[UNK]', ' ', 'e', 't', 'o', 'a', 'i', 'h', 's', 'r', 'n', '\n', 'l', 'd', 'u', 'm', 'y', 'w', ',', 'c', 'f', 'g', 'b', 'p', ':', 'k', 'v', '.', "'", ';', '?', '!', '-', 'j', 'q', 'x', 'z', '3', '&', '$']


5. Print the number of tokens in this vocabulary.

In [27]:
print(f"Number of tokens in vocabulary: {text_vec_layer.vocabulary_size()}")

Number of tokens in vocabulary: 41


6. The Text Vectorization layer reserved the value 0 for padding tokens,and it reserved 1 for unknown characters. We won’t need either of these tokens for now, so let’s subtract 2 from the character IDs and compute the number of distinct characters and the total number of characters

In [28]:
# Example code from the book. Can be used as is.
encoded -= 2 # drop tokens 0 (pad) and 1 (unknown),which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2#nbr of distinctchars = 39
dataset_size =len(encoded)# total number of chars=1,115,39
print(f"Number of distinct characters: {n_tokens}")
print(f"Total number of characters: {format(dataset_size, ',d')}")

Number of distinct characters: 39
Total number of characters: 1,115,394


7. A utility function to convert a long sequence of character IDs into adataset of input/target window pairs

In [29]:
# Example code from the book.Needs changing to use the cache parameter!
# Add caching before shuffling.
def to_dataset(sequence, length, shuffle=False, seed=None,batch_size=32, cache=True):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    # YOUR CODE HERE. Add caching
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)

    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

8. Create training, validation and test set using the code below.

In [30]:
SEQ_LENGTH = 100
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length=SEQ_LENGTH,shuffle=True, seed=42, cache=True)
valid_set = to_dataset(encoded[1_000_000:1_060_000],length=SEQ_LENGTH, cache=True)
test_set = to_dataset(encoded[1_060_000:],length=SEQ_LENGTH, cache=True)

## 1.6  Create and Train a Model

Next, create and train a model:
- Use an embedding dimension of 32, use 4 heads in each multi-headed at-tention layer, and use a stack of two decoder blocks. When you create this model you should see that it has 32 833 parameters

In [31]:
EMBED_SIZE = 32
NUM_HEADS = 4
NUM_BLOCKS = 2

In [32]:
model = get_model(n_tokens=n_tokens,
                    embed_size=EMBED_SIZE,
                    num_blocks=NUM_BLOCKS,
                    num_heads=NUM_HEADS,
                    max_seq_length=SEQ_LENGTH)
model.summary()

Building EmbeddingWithPosition with input shape (None, None)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_with_position_2       │ (None, None, 32)       │         4,448 │
│ (EmbeddingWithPosition)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gpt_decoder_block_0             │ (None, None, 32)       │        12,704 │
│ (GPTDecoderBlock)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gpt_decoder_block_1             │ (None, None, 32)       │        12,704 │
│ (GPTDecoderBlock)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, None, 39)       │         1,287 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,143 (121.65 KB)

 Trainable params: 31,143 (121.65 KB)

 Non-trainable params: 0 (0.00 B)

- Use then adam optimizer and the appropriate loss function for this problem. Remember that we didn’t use any activation function on the lastlayer. Train the model for a small couple of epochs. (Make sure to usea GPU or this will be very slow.)

In [ ]:

#! Als je niet (One-Hot encoding) categorical_crossentropy gebruikt,
#! gebruik je sparse_categorical_crossentropy.

#? Train the model on GPU if available
with tf.device('/GPU:0'):
    model.compile(
        optimizer="nadam",
        metrics=["accuracy"],
        loss="sparse_categorical_crossentropy"
        )

model.fit(train_set,
        epochs=2, # For testing purposes, set to 2. Increase for better results.
        validation_data=valid_set,
        callbacks=[keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
        )

Epoch 1/2
   2182/Unknown 157s 67ms/step - accuracy: 0.5250 - loss: 1.5299

KeyboardInterrupt: 

In [5]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## 1.7  Generate some Text
In order to be able to feed the model strings (instead of numbers), we need to add a `TextVectorizationLayer` to it. We also use a custom SubtractLayer to subtract two from the token IDs. (In the book this is done using a Lambda layer.) 

Note: Since we want to save (and restore) the model at a later point we have to jump through these hoops because of (I believe) some bugs in Keras.

In [33]:
@keras.saving.register_keras_serializable()
class SubtractLayer(keras.layers.Layer):
    def __init__(self, constant, **kwargs):
        super().__init__(**kwargs)
        self.constant = constant

    def call(self, inputs):
        return inputs - self.constant

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "constant": self.constant
        }

The code below creates a model that accepts strings as input:

In [34]:
# Create model including textvectorization layer but recreate without adapt,
# since otherwise we cannot save and load the model.
shakespeare_model = keras.Sequential([
    keras.layers.TextVectorization(split="character", standardize=None, vocabulary=text_vec_layer.get_vocabulary(
    ), name="text_vec_layer"), SubtractLayer(2),
    model # reference to the trained model
])

Consider the following method from the book:

In [35]:
# Example code from the book
def next_char(text, temperature=1):
    # shakespeare_model is the model we trained earlier
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    # text_vec_layer is the layer we adapted earlier
    return text_vec_layer.get_vocabulary()[char_id + 2]

Adapt this method to have the following signature:

In [36]:
def next_char(shakespeare_model, text, temperature=1):
    # shakespeare_model is the model we trained earlier
    y_proba = shakespeare_model.predict([text], verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    # text_vec_layer is the layer we adapted earlier
    return text_vec_layer.get_vocabulary()[char_id + 2]

This method should be completely self contained. It should not reference any global variables from the notebook. Furthermore, keep in mind that our modelreturns logits and not probabilities (as the model in the book does). Adapt the method `extend_text` from the book to be completely self-contained as well. Also, keep in mind that our model will crash if you feed it a string thatis longer than the maximum sequence length

In [37]:
# Example code from thebook.Needschanging!
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        char = next_char(shakespeare_model, text, temperature)
        text += char
    return text

In [38]:
x = keras.ops.convert_to_tensor(["First Citize"])

# Force CPU execution
with tf.device("/CPU:0"):
    out = shakespeare_model.predict([x], verbose=0).shape
print(out)
# output = next_char(shakespeare_model, x, -1)[0]

(1, 12, 39)


2025-12-07 15:11:54.706976: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


## 1.8  Save and Restore the Model
Since we were (hopefully) careful to make sure that all our custom layers couldbe saved and restored, we should be able to save the model as a `.kerasfile`. Restore the model under a different name and generate some more text fromt his restored model. In principle, you should be able to restore the model in a different notebook 3 and use the methods `next_char` and `extend_text` to generate some more text.

In [39]:
model.save("models/shakespeare_model.keras")

# 2  Text Classification using an Encoder-Only Transformer
In this exercise we will build an encoder-only transformer and apply it on a textclassification task. To better understand the model architecture, we will build and train the model (almost) from scratch.

Note: in practice, you would use a pretrained language model and possibly fine-tune it

## 2.1  Copy and Change Classes from Previous ExerciseAn encoder-only transformer is very similar to a decoder-only transformer:
- It also starts with a positional embedding. Copy the code for EmbeddingWithPosition from the previous exercise.
- The encoder blocks employ the same FeedForwardlayer as the decoderblocks.Copy the code forFeedForwardfrom the previous exercise.
- The encoder block itself is very similar to the decoder block in a decoder-only transformer. Copy the code for GPTDecoderBlock from the previous exercise. 

Make the following changes to GPTDecoderBlock:
- Rename the class to EncoderBlock.
- The multi-headed attention does not need the causal mask when it is called, since each token is allowed to attend to all tokens before ánd after it. Remove the causal mask

In [19]:
# A GPT Encoderblock (with out crossattention)
@keras.saving.register_keras_serializable()
class EncoderBlock(keras.layers.Layer):
    def __init__(self, num_heads, embed_size, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.embed_size = embed_size
        #! YOUR CODE HERE
        #! Add needed layers (either from Keras or your own custom layer)
        self.attention = keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                         key_dim=(embed_size//num_heads))
        self.normalization_1 = keras.layers.LayerNormalization()
        self.normalization_2 = keras.layers.LayerNormalization()
        self.feed_forward_network = FeedForward()

    def call(self, inputs, attention_mask=None):
        #! YOUR CODE HERE
        #! Perform the computation on inputs and return result
        skip_connection_1 = inputs  # (seq_length, embed_size)

        inputs = self.attention(inputs,
                                inputs,
                                attention_mask=attention_mask,
                                use_causal_mask=False)

        # * Output van attention
        x = self.attention(inputs, inputs, use_causal_mask=False)
        # * Output van attention en normalisatie
        x = self.normalization_1(x + skip_connection_1)

        skip_connection_2 = x
        # * Output van feedforward netwerk
        x = self.feed_forward_network(x)
        # * Output van feedforward en normalisatie
        x = self.normalization_2(x + skip_connection_2)
        return x  # (seq_length, embed_size)

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "num_heads": self.num_heads,
            "embed_size": self.embed_size
        }

## 2.2  Create an Encoder-Only Classification Model
The encoder from the original “Attention is all you Need” paper outputs contex-tual embeddings of shape:(batch_size, seq_length, embed_size).

**Note**: in principle this model will work with any sequence length, as longas it is not longer than the maximum sequence length as specified in EmbeddingWithPosition. 

We need a way of attaching a small network for  classification on top of these contextual embeddings. We will do the following:

- First, we will reduce the contextual embeddings of shape(batch_size, seq_length, embed_size)to a tensor of fixedsize by viewing these contextual embeddings as a one-dimensional se-quence and applying a one-dimensional global average pooling layerto it. This reduces the contextual embeddings to a tensor of size(batch_size, embed_size).
- Next, we add a fully connected layer withembed_sizeunits and the ReLUactivation function.
- Finally, we add a fully connected layer with the correct number of unitsand the correct activation function given the type of classification problem.Complete the following method

In [20]:
import keras

In [21]:
def get_classification_encoder_model(num_tokens:int,
                                     max_seq_length:int,
                                     embed_size:int,
                                     num_heads:int,
                                     num_blocks:int,
                                     num_classes:int,
                                     use_mask:bool=False,
                                     use_scale_embedding:bool=False):
    """
    num_tokens:the vocabulary size
    max_seq_length: maximum length of any sequence
    embed_size: the embedding dimension
    num_heads:the number of headsineachmulti-headedattention
    num_blocks:the number of encoder blocks
    num_classes:the number of classes to classify the text into
    use_mask: whether to use a mask in the encoder blocks
    """
    API_type = "FUNCTIONAL" #? "SEQUENTIAL" or "FUNCTIONAL"
    inputs = keras.layers.Input(shape=[max_seq_length], dtype=int) #? (b, seq_length)
    attention_mask = None
    mask = keras.ops.not_equal(inputs, 0)
    if use_mask:
        #? Broadicasting wordt gedaan door Keras zelf
        attention_mask = keras.ops.not_equal(inputs, 0) #* blijft zelfde: (b, seq_length)
        #? We willen naar (b, <seq_length>, seq_length)
        #? (b, 1, seq_length), later broadcasten naar (b, seq_length, seq_length)
        attention_mask = keras.ops.expand_dims(attention_mask, axis=1)

    x = EmbeddingWithPosition(num_tokens, max_seq_length, embed_size)(inputs)
    for _ in range(num_blocks):
        x = EncoderBlock(num_heads=num_heads,
                        embed_size=embed_size)(x,attention_mask=attention_mask) #!

    x = keras.layers.GlobalAveragePooling1D()(x)
    #? Embeddings op token niv -> embeddings op zin niv
    if use_scale_embedding:
        number_of_non_padding_tokens = keras.ops.sum(mask)
        scale = keras.ops.sqrt(number_of_non_padding_tokens)
        x *= scale
    #? Classificatie head
    if num_classes == 1:
        x = keras.layers.Dense(units=1, activation="sigmoid")(x)
    else:
        x = keras.layers.Dense(units=num_classes, activation="softmax")(x)

    return keras.Model(inputs=inputs, outputs=x)


In [22]:
tokens = keras.ops.convert_to_tensor([[1,3,5],[0,2,4]])
# emb_pos = EmbeddingWithPosition(num_tokens=10,
#                                     max_seq_length=5,
#                                     embed_size=32)
model = get_classification_encoder_model(num_tokens=10,
                                      embed_size=32,
                                      num_heads=4,
                                      num_blocks=10,
                                      num_classes=2,
                                      max_seq_length=3,
                                      use_mask=False)
x = model(tokens) # Should print (2,3) -> batch size

Building EmbeddingWithPosition with input shape (None, 3)


In [23]:
x.shape #? (2, 32) omdat we global average pooling hebben gedaan,
        #? Indien 2 classes (binary classification) dan (2,1)

TensorShape([2, 2])

In [24]:
import tensorflow as tf
# Create a Model
def get_model(n_tokens, embed_size,num_blocks, num_heads,max_seq_length):
    model = keras.Sequential()
    #? Voor nu op None zetten
    model.add(keras.layers.Input(shape=(None,), dtype=tf.int32))
    # YOUR CODE HERE
    # Add positional embedding layer
    model.add(EmbeddingWithPosition(n_tokens, max_seq_length, embed_size))

    for i in range(num_blocks):
        # YOUR CODE HERE
        # Add decoder blocks
        model.add(GPTDecoderBlock(num_heads=num_heads,
                                  embed_size=embed_size,
                                  name=f"gpt_decoder_block_{i}"))

    # YOUR CODE HERE
    # Add output layer.
    # Output logits, not probabilities
    model.add(keras.layers.Dense(units=n_tokens,
                                 activation="softmax",name="output_layer"))

    return model

## 2.3  Apply the Model to the IMDB-Dataset
We will now apply this model to the built-in IMDB-dataset. Load the data:

In [25]:
MAX_LEN=500
NUM_TOKENS=10_000
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=NUM_TOKENS)
print("X Shape: ", X_train.shape)
print("y Shape: ", y_train.shape)
print("X Shape: ", X_train.shape)
print("y Shape: ", y_train.shape)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
X Shape:  (25000,)
y Shape:  (25000,)
X Shape:  (25000,)
y Shape:  (25000,)


Not all reviews contain the same number of words. Use a method from `keras.utils` to pad the reviews so that they all have the same length. Usearguments so that
- padding tokens are added at the end of reviews that are shorter than `MAX_LEN`.
- reviews that are longer than `MAX_LEN` should be truncated so that the start of the review is kept

In [26]:

#? Dit is om de reviews op dezelfde lengte te krijgen
X_train = keras.utils.pad_sequences(X_train,
                                    maxlen=MAX_LEN,
                                    padding='post',
                                    truncating='post')
X_test = keras.utils.pad_sequences(X_test,
                                   maxlen=MAX_LEN,
                                   padding='post',
                                   truncating='post')

len(X_train[0])
len(X_train[1])

500

Set aside the first 10 000 examples of the test data as the validation set

In [27]:
import numpy as np
X_val = X_test[:10_000]
X_test = X_test[-10_000:]

y_val = y_test[:10_000]
y_test = y_test[-10_000:]

X_val.shape

(10000, 500)

In [28]:
NUM_CLASSES = 2
EMBED_SIZE = 32
NUM_HEADS = 4
NUM_BLOCKS = 2

In [29]:
tokens = keras.ops.convert_to_tensor([[1,3,5],[0,2,4]])
# emb_pos = EmbeddingWithPosition(num_tokens=10,
#                                     max_seq_length=5,
#                                     embed_size=32)
model = get_classification_encoder_model(num_tokens=NUM_TOKENS,
                                      embed_size=EMBED_SIZE,
                                      num_heads=NUM_HEADS,
                                      num_blocks=NUM_BLOCKS,
                                      num_classes=NUM_CLASSES,
                                      max_seq_length=MAX_LEN)

model.summary()

Building EmbeddingWithPosition with input shape (None, 500)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_with_position_4       │ (None, 500, 32)        │       336,000 │
│ (EmbeddingWithPosition)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_13 (EncoderBlock) │ (None, 500, 32)        │        12,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_block_14 (EncoderBlock) │ (None, 500, 32)        │        12,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 361,474 (1.38 MB)

 Trainable params: 361,474 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:

if NUM_CLASSES == 1:
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
else:
    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])

In [31]:
early_stopping = keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)
model.fit(X_train,
          y_train,
          epochs=10,
          validation_data=(X_val, y_val),
          callbacks=[early_stopping])

Epoch 1/10
 29/782 ━━━━━━━━━━━━━━━━━━━━ 22:19 2s/step - accuracy: 0.4838 - loss: 0.8097

KeyboardInterrupt: 

In [ ]:
zin = "Bad"
woorden = zin.split(" ")
len(woorden)

tokenizer = keras.datasets.imdb.get_word_index()

tokens = [tokenizer.get(woord.lower(), -1) for woord in woorden]
tokens


NameError: name 'keras' is not defined

In [ ]:
tokens_padded = keras.utils.pad_sequences(keras.ops.convert_to_tensor([tokens]),
                                            maxlen=MAX_LEN,
                                            padding='post',
                                            truncating='post')
tokens_padded

array([[[1, 3, 5],
        [0, 2, 4],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=int32)

In [ ]:
model.predict(tokens_padded)